# OKM Van Loon #

This script reads in an Excel file containing a Bill of Material (BOM), and an Excel file containing additional information such as prices per ingredient, waste per HF, packaging, etc. It then combines these two to calculate the cost of the products. It then outputs these costs as an Excel file.

## Set-up ##

### Imports ###

In [98]:
#import openpyxl
import pandas as pd

### Functions ###

In [99]:
def calc_cost(bom : pd.DataFrame, costs : pd.DataFrame, exclude : list, q : str, p : str) -> float:
    """ 
    calculate cost for an object given its BOM 
    
    args
    ----
    bom : DataFrame containing the BOM
    costs : DataFrame containing the costs
    exclude : BOM items to exclude
    q : name of the bom DataFrame column containing the quantity
    p : name of the costs DataFrame column containing the price

    returns
    -------
    cost : the cost of the object
    """

    cost = 0.0

    for index, row1 in bom.iterrows():
        item = row1['hf_nr']

        # filter items
        if not item.startswith(tuple(exclude)):
            for index, row2 in costs.iterrows():
                if item == str(int(row2["Rijlabels"])):
                    cost += float(row2[p]) * float(row1[q])
    
    return cost

### Objects ###

In [100]:
class recipe:
    """ a recipe """

    def __init__(self, name : str, id : str, data : pd.DataFrame, cost: float=0.0, HFs: list=[]) -> None:
        """ initialise an instance of recipe"""
        self.name = name
        self.id = id
        self.data = data
        self.cost = cost
        self.HFs = HFs

    def __str__(self) -> str:
        """ set the string representation of a recipe """
        return f'{self.id} {self.name}'
    
    def calc_recipe_cost(self) -> float:
        """ invoke the calc_cost function with the recipe parameters"""
        return calc_cost(bom=self.data, costs=cost_data, exclude=["HF", "3"], q='Aantal (Basis)', p='Gemiddelde van Prijs per Ingredient')
    
class HF:
    """ an HF (halffabricaat)"""

    def __init__(self, name: str, id: str, cost: float, weight: float, waste: float, data: pd.DataFrame) -> None:
        """ initialise an instance of HF"""
        self.name = name
        self.id = id
        self.cost = cost
        self.weight = weight
        self.waste = waste
        self.data = data
    
    def __str__(self):
        """ set the string representation of an HF"""
        return f'{self.id} {self.name}'
    
    def calc_hf_cost(self) -> float:
        """ invoke the calc_cost function with the HF parameters"""
        return calc_cost(bom=self.data, costs=cost_data, exclude=["HF", "3"], q='Aantal (Basis)', p='Gemiddelde van Prijs per Ingredient')

## Data preparation ##

### Data loading ###

#### BOM ####


In [101]:
bom_data_raw = pd.read_excel("NAV Recepten Download.xlsx", skiprows=1, header=None, decimal=",")

#### Costs ####

In [102]:
cost_data = pd.read_excel("NAV Recepten Download Extra.xlsx", sheet_name="Prijslijst", skiprows=2)

#### Packaging ####

In [103]:
packaging_data = pd.read_excel("NAV Recepten Download Extra.xlsx", sheet_name="Verpakking", skiprows=2, skipfooter=1)

### Data cleaning ###

#### BOM ####

##### Split data into recipes #####

In [104]:
recipes = []

for i in range(len(bom_data_raw)):
    # a new recipe starts
    if bom_data_raw[4][i] == 'Omschrijving':
        start_idx = i + 1
        recipe_name = bom_data_raw[4][i + 1]
        recipe_id = bom_data_raw[3][i + 1]

        # the recipe ends
        for j in range(i, len(bom_data_raw)):
            if bom_data_raw[3][j] == 'Kostenaandeel voor dit artikel':
                end_idx = j
                recipe_data = bom_data_raw.iloc[(i + 2):j].drop(range(8, 13), axis='columns').reset_index()
                recipe_data = recipe_data.rename(columns={0: "id_nr", 1: "nr", 2: "Niveau", 3: "hf_nr", 4: "Omschrijving", 5: "Aantal (Basis)", 6: "Basiseenheid", 7: "Materiaalkosten"})
                recipe_data = recipe_data.astype({"id_nr": str, "nr": int, "Niveau": int, "hf_nr": str, "Omschrijving": str, "Aantal (Basis)": float, "Basiseenheid": str, "Materiaalkosten": float})
                recipes.append(recipe(recipe_name, recipe_id, recipe_data))
                i += j
                break

##### Split recipes into HFs #####

In [105]:
for recipe in recipes:
    HFs = []
    for i in range(len(recipe.data)):
        if recipe.data["hf_nr"][i].startswith("HF"):
            HF_level = recipe.data["Niveau"][i]
            for j in range(i + 1, len(recipe.data)):
                if not recipe.data["Niveau"][j] <= HF_level:
                    j += 1
                else:
                    break
            HFs.append(HF(name=recipe.data["Omschrijving"][i], id=recipe.data["hf_nr"][i], cost=0.0, weight=0.0, waste=0.0, data=recipe.data.iloc[i + 1 : j + 1 ]))

##### Split recipes into ingredients #####
Ingredients are stored in a DataFrame, as they are not complex objects

In [106]:
ingredients = {}

for recipe in recipes:
    for i in range(len(recipe.data)):
        if not recipe.data["hf_nr"][i].startswith('HF') and not (recipe.data["hf_nr"][i].startswith("3")):
            if not recipe.data["hf_nr"][i] in ingredients.keys():
                ingredients[recipe.data["hf_nr"][i]] = {"Ingredient" : recipe.data["Omschrijving"][i], "id" : recipe.data["hf_nr"][i], "Prijs" : recipe.data["Materiaalkosten"][i], 
                                                        "Eenheid" : recipe.data["Basiseenheid"][i]}

ingredients_df = pd.DataFrame.from_dict(ingredients, orient='index').reset_index(drop=True)

## Cost calculations ##

### Recipes ###

In [107]:
for recipe in recipes:
    recipe.cost = recipe.calc_recipe_cost()

### HFs ###

In [108]:
for HF in HFs:
    HF.cost = HF.calc_hf_cost()

## Output Excel file ##

### Recipes ###

In [109]:
recipes_dict = {}

i = 0
for recipe in recipes:
    recipes_dict[i] = [recipe.name, recipe.id, recipe.cost]
    i += 1

recipes_df = pd.DataFrame.from_dict(recipes_dict, orient='index', columns=['Product', 'id', 'Kostprijs (zonder verpakking) Q1'])

### HFs ###

In [110]:
HFs_dict = {}

i = 0
for HF in HFs:
    HFs_dict[i] = [HF.name, HF.id, HF.weight, HF.waste, HF.cost]
    i += 1

HFs_df = pd.DataFrame.from_dict(HFs_dict, orient='index', columns=['Halffabricaat', 'id', 'Gewicht Q1', 'Waste Q1', 'Kostprijs (zonder verpakking) Q1'])

### BOM ###

In [111]:
frames = []
for recipe in recipes:
    frames.append(recipe.data)

BOM_df = pd.concat(frames)

### Save to Excel ###

In [113]:
with pd.ExcelWriter("Output.xlsx") as writer:
    recipes_df.to_excel(writer, sheet_name="Producten")
    HFs_df.to_excel(writer, sheet_name="Halffabricaten")
    BOM_df.to_excel(writer, sheet_name="BOM")
    ingredients_df.to_excel(writer, sheet_name="Ingrediënten")